In [164]:
import pandas as pd
import numpy as np
import os
from pathlib import Path

from evidently import ColumnMapping

from evidently.report import Report
from evidently.metrics.base_metric import generate_column_metrics
from evidently.metric_preset import DataDriftPreset, TargetDriftPreset, DataQualityPreset, RegressionPreset
from evidently.metrics import *

from evidently.test_suite import TestSuite
from evidently.tests.base_test import generate_column_tests
from evidently.test_preset import DataStabilityTestPreset, NoTargetPerformanceTestPreset, RegressionTestPreset
from evidently.tests import *

import datetime as dt

In [165]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [166]:
def csv_path(csv):
    path = Path.cwd()
    path = str(path)
    path = path[:-4]
    path = path + csv
    f_path = Path(path)
    return f_path

In [167]:
#original_dataset = pd.read_csv(r'C:\Users\A7202\Downloads\Datadrift_POC_with_health_data\artifact\data\healthcare_dataset.csv')
#original_dataset = pd.read_csv('C:/Users/A4647/OneDrive - Axtria/Desktop/Datadrift/Datadrift_POC_with_health_data/)
original_dataset = pd.read_csv(csv_path('//artifact//data//healthcare_dataset.csv'))
original_dataset

,Name,Age,Gender,Blood Type,Medical Condition,Date of Admission,Doctor,Hospital,Insurance Provider,Billing Amount,Room Number,Admission Type,Discharge Date,Medication,Test Results
0,Tiffany Ramirez,81,Female,O-,Diabetes,2022-11-17,Patrick Parker,Wallace-Hamilton,Medicare,37490.983364,146,Elective,2022-12-01,Aspirin,Inconclusive
1,Ruben Burns,35,Male,O+,Asthma,2023-06-01,Diane Jackson,"Burke, Griffin and Cooper",UnitedHealthcare,47304.064845,404,Emergency,2023-06-15,Lipitor,Normal
2,Chad Byrd,61,Male,B-,Obesity,2019-01-09,Paul Baker,Walton LLC,Medicare,36874.896997,292,Emergency,2019-02-08,Lipitor,Normal
3,Antonio Frederick,49,Male,B-,Asthma,2020-05-02,Brian Chandler,Garcia Ltd,Medicare,23303.322092,480,Urgent,2020-05-03,Penicillin,Abnormal
4,Mrs. Brandy Flowers,51,Male,O-,Arthritis,2021-07-09,Dustin Griffin,"Jones, Brown and Murray",UnitedHealthcare,18086.344184,477,Urgent,2021-08-02,Paracetamol,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,James Hood,83,Male,A+,Obesity,2022-07-29,Samuel Moody,"Wood, Martin and Simmons",UnitedHealthcare,39606.840083,110,Elective,2022-08-02,Ibuprofen,Abnormal
9996,Stephanie Evans,47,Female,AB+,Arthritis,2022-01-06,Christopher Yates,Nash-Krueger,Blue Cross,5995.717488,244,Emergency,2022-01-29,Ibuprofen,Normal
9997,Christopher Martinez,54,Male,B-,Arthritis,2022-07-01,Robert Nicholson,Larson and Sons,Blue Cross,49559.202905,312,Elective,2022-07-15,Ibuprofen,Normal
9998,Amanda Duke,84,Male,A+,Arthritis,2020-02-06,Jamie Lewis,Wilson-Lyons,UnitedHealthcare,25236.344761,420,Urgent,2020-02-26,Penicillin,Normal


In [168]:
#updated_dataset = pd.read_csv(r'C:\Users\A7202\Downloads\Datadrift_POC_with_health_data\artifact\data\healthcare_dataset_updated.csv')
#updated_dataset = pd.read_csv('C:/Users/A4647/OneDrive - Axtria/Desktop/Datadrift/Datadrift_POC_with_health_data/artifact/data/healthcare_dataset_updated.csv')
updated_dataset = pd.read_csv(csv_path('//artifact//data//healthcare_dataset_updated.csv'))
updated_dataset.tail(10)

,Name,Age,Gender,Blood Type,Medical Condition,Date of Admission,Doctor,Hospital,Insurance Provider,Billing Amount,Room Number,Admission Type,Discharge Date,Medication,Test Results,Body_temperature
10339,Michelle Harris,74.0,Female,B+,Cancer,04-02-21,David Patel,Lin Inc,Medicare,31303.64787,284,Elective,22-02-21,Penicillin,Inconclusive,96
10340,Julie Mckenzie,35.0,Female,A+,Obesity,07-02-19,Jerome Davis,Duarte Inc,Medicare,31303.64787,242,Urgent,22-02-19,Penicillin,Normal,98
10341,Robert Ho,47.0,Female,AB+,Cancer,08-12-18,Michael Villanueva,"Mason, Baker and Smith",Cigna,31303.64787,209,Urgent,15-12-18,Ibuprofen,Abnormal,94
10342,Katelyn Vincent,29.0,Female,AB+,Asthma,11-12-19,Elizabeth Chase,Parker Inc,Aetna,31303.64787,404,Elective,13-12-19,Penicillin,Abnormal,96
10343,Anne Wallace,18.0,Female,AB+,Arthritis,30-07-19,Edward Collins,Wilkinson-Mcdonald,Aetna,31303.64787,442,Emergency,28-08-19,Paracetamol,Normal,92
10344,Joseph Lynch,84.0,Female,AB+,Diabetes,08-10-21,Jonathon Klein,Garza-Everett,Medicare,31303.64787,110,Urgent,01-11-21,Paracetamol,Normal,92
10345,Dr. Leah Delgado,42.0,Female,AB+,Asthma,02-10-21,David Sanchez,"Edwards, Salinas and Snyder",Cigna,31303.64787,226,Elective,07-10-21,Penicillin,Inconclusive,94
10346,Melissa Robinson,40.0,Female,AB+,Obesity,29-03-23,Matthew Downs,Ellis-Davis,Cigna,31303.64787,400,Urgent,14-04-23,Penicillin,Inconclusive,94
10347,Carol Murray,77.0,Female,AB+,Diabetes,06-02-22,Jennifer Acosta,Miles Group,Aetna,31303.64787,433,Emergency,21-02-22,Penicillin,Normal,98
10348,Nathan Elliott,29.0,Female,AB+,Diabetes,25-11-21,Karen Collier,English-Davis,Aetna,31303.64787,116,Elective,24-12-21,Aspirin,Abnormal,92


In [169]:
#insight = pd.read_csv('C:/Users/A4647/OneDrive - Axtria/Desktop/Datadrift/Datadrift_POC_with_health_data/artifact/insights/insight_check.csv')
insight = pd.read_csv(csv_path('//artifact//insights//insight_check.csv'))
insight

,no._of_rows,no._of_columns,Missing_value,Age_mean,Age_std,Room Number_mean,Room Number_std,Billing Amount,Billing Amount_std
0,10000,15,0,51.4522,19.588974,300.082,115.806027,25516.806778,14067.292709


In [170]:
if updated_dataset.shape[0] != insight['no._of_rows'][0] or updated_dataset.shape[1] != insight['no._of_rows'][1]:
    print("some differnce")
    if updated_dataset['Age'].std() != insight['Age_std'][0]:
        print("evidently")
        print("send mail function here")
        report = Report(metrics=[
            DataDriftPreset(), 
        ])
        report.run(reference_data=original_dataset, current_data=updated_dataset)
        #report.save_html(r'C:\Users\A7202\Downloads\Datadrift_POC_with_health_data\artifact\result\drift_result.html')
        path = str(csv_path('//artifact//result//drift_result.html'))
        #report.save_html('C:/Users/A4647/OneDrive - Axtria/Desktop/Datadrift/Datadrift_POC_with_health_data/artifact/result/drift_result.html')
        report.save_html(path)
        tests = TestSuite(tests=[
            TestNumberOfColumnsWithMissingValues(),
            TestNumberOfRowsWithMissingValues(),
            TestNumberOfConstantColumns(),
            TestNumberOfDuplicatedRows(),
            TestNumberOfDuplicatedColumns(),
            TestColumnsType(),
            TestNumberOfDriftedColumns(),
        ])
        path = str(csv_path('//artifact//result//testsuite.html'))
        tests.save_html(path)
        tests.run(reference_data=original_dataset, current_data=updated_dataset)
        print("report uploaded") 
        
else:
    print("no")


some differnce
evidently
send mail function here


report uploaded


In [148]:
csv_path('//artifact//result//drift_result.html')

WindowsPath('c:/Users/A4647/OneDrive - Axtria/Desktop/Datadrift/Datadrift_POC_with_health_data/artifact/result/drift_result.html')